<a href="https://colab.research.google.com/github/primojr/estudos_python/blob/master/py/rede_neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo:

## Treinar o modelo usando `MLPClassifier` e comprar o resultado do `randon_forest`

## 1.Importar bibliotecas


In [ ]:
from sklearn import datasets 
from sklearn.neural_network import  MLPClassifier 

# # Bibliotecas Auxiliares
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(13)

df = pd.read_csv("dfFAmostra.csv", delimiter=";")

# # TensorFlow e tf.keras
# import tensorflow as tf
# from tensorflow import keras


# print(tf.__version__)

In [ ]:
df.head(2)

# 2. Ajustes na base


In [ ]:
# Sys
df['system'] = df['system'].replace('android','1')
df['system'] = df['system'].replace('ios','0')
df['system'] = df['system'].astype(float)

# # Classe cep
df['class_cep'].value_counts()
df['class_cep'] = df['class_cep'].replace('A.Ruim','0')
df['class_cep'] = df['class_cep'].replace('B.Neutro','1')
df['class_cep'] = df['class_cep'].replace('C.Medio','2')
df['class_cep'] = df['class_cep'].replace('D.Bom','3')
df['class_cep'] = df['class_cep'].replace('E.MuitoBom','4')
df['class_cep'] = df['class_cep'].astype(float)


## Binario resposta
df.loc[df['classe1'] == 'ComCartaoApp', 'Class'] = 1
df.loc[df['classe1'] != 'ComCartaoApp', 'Class'] = 0
df = df.drop('classe1',axis=1)
df.head()


# Remover na()
df = df.dropna()
df = df.reset_index()

#I Tipos dos dados
df.dtypes

# 3. Separar amostra e treino


In [ ]:
#df = df.loc[:,df.columns[1:]]
X = df.loc[:, df.columns[2:14]]
y = df.loc[:,['Class']]

n_aleatorio = np.arange(df.shape[0])
np.random.shuffle(n_aleatorio)

X_train = X.loc[n_aleatorio[:3000],:]
X_test  = X.loc[n_aleatorio[3000:],:]

y_train = y.loc[n_aleatorio[:3000],:]
y_test  = y.loc[n_aleatorio[3000:],:]

# 4. Treino do modelo

## 4.2 MPLClassifier

-   Paramentros: -hidden_layer_sizes =
    -   activation = Função de ativação\
    -   max_inter = Maximo de interação\
    -   learning_rate_init =


In [ ]:
from pandas.core.common import random_state
clf = MLPClassifier(solver = 'adam',
                    hidden_layer_sizes=(2,) ,
                    learning_rate_init=0.00001, 
                    activation='logistic',
                    max_iter = 1500,
                    random_state = 1
                    )

clf.fit(X_train, y_train)

y_pred = clf.predict_proba(X_test)
y_pred

### 4.2.1 Validar ajuste


In [ ]:
# Categorizar a Classe de previsão
y_linha = pd.DataFrame(y_pred)
y_linha.loc[y_linha[1] >= .5, 'Class'] = 1
y_linha.loc[y_linha[1] <  .5, 'Class'] = 0 

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_linha['Class'])

## 4.2 Treino usando keras


In [ ]:
import tensorflow as tf
from tensorflow import keras
tf.random.set_seed(42)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss = tf.keras.losses.binary_crossentropy,
    optimizer = tf.keras.optimizers.Adam(lr=0.001),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

history = model.fit(X_train, y_train, epochs=100)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize = (8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
# Categorizar a Classe de previsão
y_linha = pd.DataFrame(y_pred)
y_linha.loc[y_linha[0] >= .5, 'Class'] = 1
y_linha.loc[y_linha[0] <  .5, 'Class'] = 0 

#from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_linha['Class'])